In [1]:
import torch
print(torch.cuda.is_available())

True


In [3]:
# Obtain Dataset
!wget http://cs231n.stanford.edu/tiny-imagenet-200.zip
!unzip -qq 'tiny-imagenet-200.zip'

--2021-10-09 01:21:11--  http://cs231n.stanford.edu/tiny-imagenet-200.zip
Resolving cs231n.stanford.edu (cs231n.stanford.edu)... 171.64.68.10
Connecting to cs231n.stanford.edu (cs231n.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248100043 (237M) [application/zip]
Saving to: ‘tiny-imagenet-200.zip’

tiny-imagenet-200.z 100%[===================>] 236.61M  52.1MB/s    in 5.0s    

2021-10-09 01:21:16 (47.6 MB/s) - ‘tiny-imagenet-200.zip’ saved [248100043/248100043]



# Loading Data Set and Assembling Training, Validation, and Test Sets

* Taken from Previous Coding Assignment - week-4-opt-2

In [48]:
# Load training dataset, then split into training, validation, and test sets
import torch
import torchvision
import torchvision.transforms as transforms

BATCH_SIZE = 16

# Number of batches in Training, Validation, Test Sets
TRAINING_BATCHES = 100
VALIDATION_BATCHES = 50
TEST_BATCHES = 50

# Load training data
train_dir = "./tiny-imagenet-200/train"
train_data = torchvision.datasets.ImageFolder(train_dir, transform=transforms.Compose([transforms.ToTensor()]))
train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=BATCH_SIZE, shuffle = True)


# Loop through data loader and add batches and labels based on constants
training_set = []
validation_set = []
test_set = []
i = 0
for set_data in train_data_loader:
  i += 1
  if i <= TRAINING_BATCHES:
    training_set.append(set_data)
  
  elif i <= TRAINING_BATCHES + VALIDATION_BATCHES:
    validation_set.append(set_data)
  
  elif i <= TRAINING_BATCHES + VALIDATION_BATCHES + TEST_BATCHES:
    test_set.append(set_data)
  
  else:
    break

print(len(training_set))
print(len(validation_set))
print(len(test_set))

100
50
50


In [ ]:
!pip install perceiver-pytorch

In [88]:
import torch
from perceiver_pytorch import PerceiverIO

original_model = PerceiverIO(
    dim = 64,                    # dimension of sequence to be encoded
    queries_dim = 1,            # dimension of decoder queries
    logits_dim = 200,            # dimension of final logits
    depth = 6,                   # depth of net
    num_latents = 256,           # number of latents, or induced set points, or centroids. different papers giving it different names
    latent_dim = 512,            # latent dimension
    cross_heads = 1,             # number of heads for cross attention. paper said 1
    latent_heads = 8,            # number of heads for latent self attention, 8
    cross_dim_head = 64,         # number of dimensions per cross attention head
    latent_dim_head = 64,        # number of dimensions per latent self attention head
    weight_tie_layers = False    # whether to weight tie layers (optional, as indicated in the diagram)
)

seq = torch.randn(3, 64, 64)
queries = torch.rand(3, 64, 1)

logits = original_model(seq, queries = queries) # (1, 128, 100) - (batch, decoder seq, logits dim)
print(logits.squeeze().size())


torch.Size([3, 64, 200])


In [93]:
LR = 1
MOMENTUM = 0.9
WEIGHT_DECAY = 1e-6
EPOCHS = 8

def train_model(model_in, learning_rate = LR, weight_d = WEIGHT_DECAY, momentum = MOMENTUM):
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  model = model_in
  model.to(device)
  
  # Define Loss Function and get optimizer
  train_loss = {}
  validation_acc = {}
  loss_f = torch.nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(), lr = LR, weight_decay=WEIGHT_DECAY, betas=(momentum, 0.999))
  epoch_loss = {}
  for epoch in range (EPOCHS):
    print("EPOCH : ", epoch)
    model.train()
    batch_loss = []

    # Iterate through training set, collect loss values and update model
    for im, truth_labels in training_set:
      im = im.mean(1)
      # print(im.size())
      im = im.to(device)
      queries = torch.randn(BATCH_SIZE, 64, 1)
      # queries[:, 0, 0] = truth_labels[:]
      # print(queries, truth_labels.size())
      queries = queries.to(device)
      truth_labels = truth_labels.to(device)
      predicted_labels = model(im, queries = queries).mean(1).squeeze()
      predicted_labels = predicted_labels.to(device)
      print(predicted_labels.size())
      loss = loss_f(predicted_labels, truth_labels)
      loss.backward()
      optimizer.step()
      optimizer.zero_grad()
      batch_loss.append(loss.item())
      accuracy = (predicted_labels.argmax(1) == truth_labels).float().mean().item()
      print("******Train ACCURACY****** : ", accuracy)

    
    print("TRAIN_LOSS of Each Batch: ", torch.FloatTensor(batch_loss).mean().item())
    epoch_loss[epoch] = batch_loss

    # Iterate through validation set and compute validation accuracy
    model.eval()
    accuracies = []
    for validation_im, validation_labels in validation_set:
      validation_im = validation_im.to(device)
      validation_im = validation_im.mean(1)
      validation_labels = validation_labels.to(device)
      queries = torch.randn(BATCH_SIZE, 1, 1)
      queries[:, 0, 0] = validation_labels[:]
      queries = queries.to(device)
      predicted_labels = model(validation_im, queries = queries).squeeze().argmax(1)
      accuracy = (predicted_labels == validation_labels).float().mean().item()
      accuracies.append(accuracy)
    
    validation_set_accuracy = torch.FloatTensor(accuracies).mean().item()
    print("******VALIDATION ACCURACY****** : ", validation_set_accuracy)
    validation_acc[epoch] = validation_set_accuracy

  # # Calculate test set accuracy
  # model.eval()
  # accuracies = []
  # for test_im, test_labels in test_set:
  #     test_im = test_im.to(device)
  #     test_labels = test_labels.to(device)

  #     predicted_labels = model(test_im).argmax(1)
  #     accuracy = (predicted_labels == test_labels).float().mean().item()
  #     accuracies.append(accuracy)
    
  # test_set_accuracy = torch.FloatTensor(accuracies).mean().item()
  # print("******TEST ACCURACY****** : ", test_set_accuracy)
  # return train_loss, validation_acc, test_set_accuracy

In [94]:
train_model(original_model)

EPOCH :  0
torch.Size([8, 200])
******Train ACCURACY****** :  0.0
torch.Size([8, 200])
******Train ACCURACY****** :  0.0
torch.Size([8, 200])
******Train ACCURACY****** :  0.0
torch.Size([8, 200])
******Train ACCURACY****** :  0.0
torch.Size([8, 200])
******Train ACCURACY****** :  0.0
torch.Size([8, 200])
******Train ACCURACY****** :  0.0
torch.Size([8, 200])
******Train ACCURACY****** :  0.0
torch.Size([8, 200])
******Train ACCURACY****** :  0.0
torch.Size([8, 200])
******Train ACCURACY****** :  0.0
torch.Size([8, 200])
******Train ACCURACY****** :  0.0
torch.Size([8, 200])
******Train ACCURACY****** :  0.0
torch.Size([8, 200])
******Train ACCURACY****** :  0.0
torch.Size([8, 200])
******Train ACCURACY****** :  0.0
torch.Size([8, 200])
******Train ACCURACY****** :  0.0
torch.Size([8, 200])
******Train ACCURACY****** :  0.0
torch.Size([8, 200])
******Train ACCURACY****** :  0.0
torch.Size([8, 200])
******Train ACCURACY****** :  0.0
torch.Size([8, 200])
******Train ACCURACY****** :  0.0

KeyboardInterrupt: ignored